In [2]:
!pip install -q transformers

In [3]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [7]:
# Mount Google Drive if model is saved there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Load model and tokenizer from saved directory
model_path = '/content/drive/MyDrive/sentiment140_model'
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

In [9]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [10]:
# Define classification function with proper preprocessing
def classify_text(text):
    # Add the same prefix used during training
    input_text = f"TEXT1: {text}"

    # Tokenize with same parameters as training
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    return probabilities[0].cpu().tolist()

In [11]:
# Class labels mapping
class_names = {
    0: "negative",
    1: "positive"
}

In [12]:
# Interactive classification loop
print("Enter text to classify (or 'quit' to exit):")
while True:
    user_input = input("> ")

    if user_input.lower() in ['quit', 'exit']:
        print("Goodbye!")
        break

    probabilities = classify_text(user_input)

    print("\nClassification results:")
    for i, prob in enumerate(probabilities):
        print(f"{class_names[i]}: {prob:.2%}")

    predicted_class = torch.tensor(probabilities).argmax().item()
    print(f"\nTop prediction: {class_names[predicted_class]} ({probabilities[predicted_class]:.2%})\n")

Enter text to classify (or 'quit' to exit):
> i feel horny

Classification results:
negative: 98.85%
positive: 1.15%

Top prediction: negative (98.85%)

> sarkar girao

Classification results:
negative: 37.11%
positive: 62.89%

Top prediction: positive (62.89%)

> the

Classification results:
negative: 50.58%
positive: 49.42%

Top prediction: negative (50.58%)

> rishabh

Classification results:
negative: 6.03%
positive: 93.97%

Top prediction: positive (93.97%)

> mario

Classification results:
negative: 14.82%
positive: 85.18%

Top prediction: positive (85.18%)

> quit
Goodbye!
